In [1]:
import sys
sys.path.append('d:\\ActivityP')

In [8]:
import Activity_pstructure

In [9]:
import sys
sys.path.append(r"d:\\ActivityP")
import Activity_pstructure

In [3]:
import pandas as pd
import sys
from pymongo.mongo_client  import MongoClient
from logging import getLogger
from Activity_pstructure.logging.logger import logger
from Activity_pstructure.exception.exception import ActivityException
from Activity_pstructure.entity.config_entity import MongoDBConfig

logger = getLogger("activity_logger")

class DataIngestion:
    def __init__(self, config: MongoDBConfig):
        self.config = config

    def fetch_data_from_mongodb(self) -> pd.DataFrame:
        try:
            logger.info("Starting data retrieval from MongoDB Atlas...")

            client = MongoClient(self.config.mongo_url)
            db = client[self.config.database_name]
            collection = db[self.config.collection_name]

            cursor = collection.find()
            data = list(cursor)

            for doc in data:  # Remove _id field if present
                doc.pop('_id', None)

            df = pd.DataFrame(data)
            logger.info(f"Retrieved {len(df)} records from MongoDB.")
            return df

        except Exception as e:
            raise ActivityException(e, sys)


In [4]:
import pandas as pd
import sys
import os
from pymongo.mongo_client  import MongoClient
from logging import getLogger
from Activity_pstructure.exception.exception import ActivityException
from Activity_pstructure.entity.config_entity import MongoDBConfig
from Activity_pstructure.constant.database import DATABASE_NAME, COLLECTION_NAME
from Activity_pstructure.components.ingestion import DataIngestion
MONGO_DB_URL = os.getenv("MONGO_DB_URL")

config = MongoDBConfig(
    mongo_url=MONGO_DB_URL,
    database_name=DATABASE_NAME,
    collection_name=COLLECTION_NAME
)

ingestion = DataIngestion(config)
df = ingestion.fetch_data_from_mongodb()
print(df.head())

           Id ActivityDate  TotalSteps  TotalDistance  TrackerDistance  \
0  1503960366    4/20/2016       10544           6.68             6.68   
1  1503960366    4/27/2016       18134          12.21            12.21   
2  1503960366     5/7/2016       11992           7.71             7.71   
3  1624580081    4/17/2016        6175           4.06             4.06   
4  1624580081     5/5/2016        2470           1.61             1.61   

   LoggedActivitiesDistance  VeryActiveDistance  ModeratelyActiveDistance  \
0                       0.0                1.96                      0.48   
1                       0.0                6.40                      0.41   
2                       0.0                2.46                      2.12   
3                       0.0                1.03                      1.52   
4                       0.0                0.00                      0.00   

   LightActiveDistance  SedentaryActiveDistance  VeryActiveMinutes  \
0                 4.24

In [5]:
def save_dataframe_to_artifacts(df: pd.DataFrame, output_dir: str, output_filename: str):
        try:
            os.makedirs(output_dir, exist_ok=True)
            file_path = os.path.join(output_dir, output_filename)

            df.to_csv(file_path, index=False)
            logger.info(f"DataFrame saved to {file_path}")

        except Exception as e:
            logger.error(f"Failed to save DataFrame: {e}")
        raise ActivityException(e, sys)